In [1]:
# Generic
import typing
import os
from pathlib import Path
import typing
import json

# Plotting
import matplotlib.pyplot as plt

# Numerical Computing
import numpy as np
import torch

# muutils
from zanj.zanj import ZANJ, ZANJ_GLOBAL_DEFAULTS

# Our Code
from muutils.nbutils.configure_notebook import configure_notebook
from maze_transformer.training.config import ConfigHolder, ZanjHookedTransformer, BaseGPTConfig, TrainConfig
from maze_dataset import MazeDataset, MazeDatasetConfig
from maze_dataset.dataset.configs import MAZE_DATASET_CONFIGS
from maze_dataset.generation import LatticeMazeGenerators
from maze_transformer.training.train_model import TrainingResult, train_model
from maze_transformer.training.wandb_logger import WandbProject


In [2]:
# set global defaults for ZANJ
ZANJ_GLOBAL_DEFAULTS.external_array_threshold = 1024
ZANJ_GLOBAL_DEFAULTS.external_list_threshold = 1024

# paths
PATH_EXAMPLES: Path = Path("../examples/")
PATH_DATA: Path = Path("../data/")

# reproducibility and device
DEVICE = configure_notebook(seed=42, dark_mode=True)
print(f"{DEVICE = }")

DEVICE = device(type='cuda')


In [3]:
print(f"{list(MAZE_DATASET_CONFIGS.keys()) = }")

# if you want to specify a custom config, you can do so here
# CFG_CUSTOM: ConfigHolder = ConfigHolder(
#     name = "custom",
#     dataset_cfg = MazeDatasetConfig(
# 		name="custom-dataset",
# 		grid_n=6,
# 		n_mazes=10000,
# 		maze_ctor=LatticeMazeGenerators.gen_dfs,
# 	),
#     model_cfg = BaseGPTConfig(
#         name="custom-model",
#         act_fn="gelu",
#         d_model=8,
#         d_head=4,
#         n_layers=2,
#     ),
#     train_cfg = TrainConfig(
#         name="custom-train",
#         optimizer=torch.optim.RMSprop,
#         optimizer_kwargs=dict(lr=0.0001),
#         batch_size=16,
#         dataloader_cfg=dict(
#             shuffle=True,
#             num_workers=0,
#             drop_last=False,
#         ),
#         intervals_count=dict(
#             print_loss=100,
#             checkpoint=5,
#             eval_fast=10,
#             eval_slow=5,
#         )
#     ),
# )

CFG_HALLWAY: ConfigHolder = ConfigHolder(
    name = "hallway-medium",
    dataset_cfg = MazeDatasetConfig(
		name="custom-hallway",
		grid_n=8,
		n_mazes=3_000_000,
		maze_ctor=LatticeMazeGenerators.gen_dfs,
        maze_ctor_kwargs=dict(
            do_forks=False,
        ),
        applied_filters=[{'name': 'collect_generation_meta', 'args': (), 'kwargs': {}}],
        seq_len_max=256,
	),
    model_cfg = BaseGPTConfig(
        name="custom-model",
        act_fn="gelu",
        d_model=128,
        d_head=32,
        n_layers=6,
    ),
    train_cfg = TrainConfig(
        name="custom-train",
        optimizer=torch.optim.AdamW,
        optimizer_kwargs=dict(lr=0.001),
        batch_size=32,
        dataloader_cfg=dict(
            shuffle=False,
            num_workers=8,
            drop_last=False,
        ),
        intervals_count=dict(
            print_loss=100,
            checkpoint=20,
            eval_fast=100,
            eval_slow=50,
        ),
        validation_dataset_cfg=100,
    ),
)

list(MAZE_DATASET_CONFIGS.keys()) = ['test-g3-n5-a_dfs-h75556', 'demo_small-g3-n100-a_dfs-h88371', 'demo-g6-n10K-a_dfs-h30615']


In [4]:
# # this is for training a "real" demo model
# CFG_DEMO: ConfigHolder = ConfigHolder.get_config_multisource(
#     cfg_names=("test-g3-n5-a_dfs-h75556", "tiny-v1", "sweep-v1"),
# )

# # this is smaller, for testing
# CFG_TEST: ConfigHolder = ConfigHolder.get_config_multisource(
#     cfg_names=("demo_small-g3-n100-a_dfs-h88371", "nano-v1", "test-v1"),
# )

In [5]:
# here is where to specify which config to actually use
CFG: ConfigHolder = CFG_HALLWAY

In [6]:
print(json.dumps(CFG.summary(), indent=2))

{
  "name": "hallway-medium",
  "dataset_cfg": {
    "name": "custom-hallway",
    "fname": "custom-hallway-g8-n3.0M-a_dfs-h66295",
    "sdc_hash": 38558245839505946605550258446345100654876335964881215358923857434720332766295,
    "seed": 42,
    "seq_len_min": 1,
    "seq_len_max": 256,
    "applied_filters": [
      {
        "name": "collect_generation_meta",
        "args": [],
        "kwargs": {}
      }
    ],
    "grid_n": 8,
    "grid_shape": [
      8,
      8
    ],
    "n_mazes": 3000000,
    "maze_ctor_name": "gen_dfs",
    "maze_ctor_kwargs": {
      "do_forks": false
    }
  },
  "model_cfg": {
    "name": "custom-model",
    "act_fn": "gelu",
    "d_model": 128,
    "d_head": 32,
    "n_layers": 6,
    "weight_processing": {
      "are_layernorms_folded": false,
      "are_weights_processed": false
    },
    "n_heads": 4
  },
  "train_cfg": {
    "name": "custom-train",
    "optimizer": "AdamW",
    "optimizer_kwargs": {
      "lr": 0.001
    },
    "batch_size": 32,
 

In [7]:
# get just the dataset, generating it if needed. 
# This step can be skipped if you set `do_generate_dataset=True` when calling `train_model`
# or if the dataset in question already exists

# load the dataset
DATASET: MazeDataset = MazeDataset.from_config(CFG.dataset_cfg, verbose=True, local_base_path=PATH_DATA)

trying to get the dataset 'custom-hallway-g8-n3.0M-a_dfs-h66295'
seeing if we can download the dataset...
no download found, or download failed
generating dataset...


generating & solving mazes: 100%|██████████| 3000000/3000000 [1:00:22<00:00, 828.27maze/s]


saving dataset to ..\data\custom-hallway-g8-n3.0M-a_dfs-h66295.zanj
Got dataset custom-hallway with 3000000 items. output.cfg.to_fname() = 'custom-hallway-g8-n3.0M-a_dfs-h66295'


In [8]:
result: TrainingResult = train_model(
	base_path=PATH_DATA,
    cfg=CFG,
	wandb_project=WandbProject.UNDERSTANDING_SEARCH, # change this to WandbProject.DEMO_NOTEBOOKS!
	do_generate_dataset=False,
	dataset_verbose=True,
    dataset=DATASET,
)

2023-10-02 04:50:44 ERROR Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


wandb: Currently logged in as: miv. Use `wandb login --relogin` to force relogin


2023-10-02 04:50:47 INFO config ={'__format__': 'ConfigHolder(SerializableDataclass)', 'dataset_cfg': {'__format__': 'MazeDatasetConfig(SerializableDataclass)', 'name': 'custom-hallway', 'seq_len_min': 1, 'seq_len_max': 256, 'seed': 42, 'applied_filters': [{'name': 'collect_generation_meta', 'args': (), 'kwargs': {}}], 'grid_n': 8, 'n_mazes': 3000000, 'maze_ctor': {'__name__': 'gen_dfs', '__module__': 'maze_dataset.generation.generators', '__doc__': ['generate a lattice maze using depth first search, iterative', '', '        # Arguments', '        - `grid_shape: Coord`: the shape of the grid', '        - `lattice_dim: int`: the dimension of the lattice', '          (default: `2`)', '        - `accessible_cells: int | float |None`: the number of accessible cells in the maze. If `None`, defaults to the total number of cells in the grid. if a float, asserts it is <= 1 and treats it as a proportion of **total cells**', '            (default: `None`)', '        - `max_tree_depth: int | floa

MemoryError: 